gonna preprocess the birds dataset for easier use

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

In [3]:
birds_full = pd.read_csv('source_data/birds.csv', sep="\t")
birds_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977813 entries, 0 to 977812
Data columns (total 50 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   gbifID                            977813 non-null  int64  
 1   datasetKey                        977813 non-null  object 
 2   occurrenceID                      977813 non-null  object 
 3   kingdom                           977813 non-null  object 
 4   phylum                            977813 non-null  object 
 5   class                             977813 non-null  object 
 6   order                             977813 non-null  object 
 7   family                            977813 non-null  object 
 8   genus                             977813 non-null  object 
 9   species                           977746 non-null  object 
 10  infraspecificEpithet              0 non-null       float64
 11  taxonRank                         977813 non-null  o

In [4]:
birds = birds_full[["species", "decimalLatitude", "decimalLongitude", "countryCode", "year", "recordedBy"]]
birds = birds.dropna()
birds = birds[birds["year"] >= 2022].drop("year", axis=1)
birds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 926672 entries, 0 to 977812
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   species           926672 non-null  object 
 1   decimalLatitude   926672 non-null  float64
 2   decimalLongitude  926672 non-null  float64
 3   countryCode       926672 non-null  object 
 4   recordedBy        926672 non-null  object 
dtypes: float64(2), object(3)
memory usage: 42.4+ MB


# 80/20

In [5]:
spec = birds.groupby("species").size().sort_values(ascending=False)
for i in [10000, 1000, 100, 10, 1]:
    print(f">{i}", spec[spec >= i].count())

>10000 26
>1000 129
>100 455
>10 1246
>1 2991


In [6]:
(birds.countryCode == "GB").sum() / birds.shape[0]

0.7765779045875995

In [7]:
top_100_GB = set(birds[birds.countryCode == "GB"].groupby("species").size().sort_values(ascending=False).head(100).index)
top_100_not_GB = set(birds[birds.countryCode != "GB"].groupby("species").size().sort_values(ascending=False).head(100).index)

len(top_100_GB & top_100_not_GB)

45

In [8]:
birds_top = birds[birds.species.isin(top_100_GB)]
birds_top.to_csv("gen_data/birds_top.csv", index=False)